In [147]:
import math
import numpy as np
from matplotlib import pyplot as plot
import random

In [148]:
def generate_random(n1,n2):
    if n1>n2:
        print('first value should be smaller')
        return 0
    r = random.uniform(0,1)
    return n1+(n2-n1)*r

In [738]:
def f_of_x1(x):
    # Rosenbrock function
    s=0
    for i in range(len(x)-1):
        s+=100*(x[i+1]-(x[i]**2))**2+(x[i]-1)**2
    return s

def f_of_x2(x):
    # Beale function
    if len(x)!=2:
        print('invalid length')
        return            
    return ((1.5-x[0]+x[0]*x[1])**2)+((2.25-x[0]+x[0]*(x[1]**2))**2)+((2.625-x[0]+x[0]*(x[1]**3))**2)

def f_of_x3(x):
    # Easom function
    if len(x)!=2:
        print('invalid length')
        return            
    return -math.cos(x[0])*math.cos(x[1])*math.exp(-(((x[0]-math.pi)**2)+((x[1]-math.pi)**2)))

def f_of_x4(x):
    # Booth function
    if len(x)!=2:
        print('invalid length')
        return            
    return ((x[0]+2*x[1]-7)**2)+((2*x[0]+x[1]-5)**2)

def f_of_x5(x):
    s = 0
    for i in range(0,5):
        s+=(x[i]-i)**2
        return s

In [752]:
def init_optimization(f_num):
    if f_num==1:
        hi = 100
        lo = -100
        dim = 4
        f = f_of_x1
        print('Rosenbrock Function in 4 dimension')
        print('Global minimum f(x_i) = 0 for x_i in [-inf,inf]')
    elif f_num==2:
        hi = 5
        lo = -5
        dim = 2
        f = f_of_x2
        print('Beale function in 2 dimension')
        print('Global minimum f(3,.5) = 0 for x,y in [-5,5]')
    elif f_num==3:
        hi = 10
        lo = -10
        dim = 2
        f = f_of_x3
        print('Eason Function in 2 dimension')
        print('Global minimum f(pi,pi) = -1 for x,y in [-10,10]')
    elif f_num==4:
        hi = 10
        lo = -10
        dim = 2
        f = f_of_x4
        print('Booth Function in 2 dimension')
        print('Global minimum f(1,3) = 0 for x,y in [-10,10]')
    elif f_num==5:
        hi = 100
        lo = 0
        dim = 5
        f = f_of_x5
        print('generated Function in 5 dimension')
        print('Global minimum f(0,1,2,3,4) = 0 for x_i in [-10,10]')
    else:
        print('provide proper value for function to be optimized')
    return hi,lo,dim, f

In [753]:
#parameters for nelder_mead algorithm
a_r = 1
a_e = 2
a_c = 0.5
a_s = 0.5

In [764]:
def generate_init(n,dim,lo,hi,f_of_x):
    x = np.empty(shape=(n,dim),dtype='float') 
    f = np.empty(shape=(n,),dtype='float')
    for i in range(n):
        for j in range(dim):
            x[i][j]=generate_random(lo,hi)
        f[i] = f_of_x(x[i])
    return x,f

In [769]:
def centroid(x):
    return (x.sum(axis=0))/len(x)
# order 
def order(x,f):
    sorted_idx = np.argsort(f,axis=0)
    x_c = centroid(x)
#    print('step 1: ordering')
    return sorted_idx, x_c
# reflection 
def reflect(x,x_c,idx):
#    print('reflection')
    return x_c + a_r*(x_c-x[idx[len(idx)-1]])
#expansion
def expand(x,x_c,x_r):
#    print('expansion')
    return x_c + a_e*(x_r-x_c)
#contraction
def contract(x,x_c,idx):
#    print('contraction')
    return x_c + a_c*(x[idx[len(idx)-1]]-x_c)
#shrink
def shrink(x,idx):
#    print('shrinking')
    for i in range(1,len(idx)-1):
        x[i]=x[0]+a_s*(x[idx[i]]-x[0])
    return x

def f_val(x, f_of_x,n):
    f = np.empty(shape=(n,),dtype='float')
    for i in range(0,n):
        f[i] = f_of_x(x[i])
    return f

In [772]:
def nelder_mead(N,n,dim,hi,lo, f_of_x,display_on):
    x,f= generate_init(n,dim,lo,hi,f_of_x)
    for itr in range(N):
        r,c=x.shape
        f = f_val(x,f_of_x,n)
        idx, x_c = order(x,f)
        if display_on==1:
            if np.mod(itr,100)==0:
                print('iteration :', itr,' cost function:', f[idx[0]])
        f_best = f[idx[0]]    

        x_r = reflect(x,x_c,idx)

        if (f_of_x(x_r) >= f_of_x(x[idx[0]])) and (f_of_x(x_r) < f_of_x(x[idx[n-1]])):
            x[idx[n-1]]=x_r
            continue

        elif f_of_x(x_r)< f_of_x(x[idx[0]]):
            x_e = expand(x,x_c,x_r)
            if f_of_x(x_e) < f_of_x(x_r):
                x[idx[n-1]]=x_e
                continue
            else:
                x[idx[n-1]]=x_r
                continue

        elif f_of_x(x_r)>= f_of_x(x[idx[n-2]]):
            x_contract = contract(x,x_c,idx)
            if f_of_x(x_contract) < f_of_x(x[idx[n-1]]):
                x[idx[n-1]] = x_contract
                continue
        else:
            x = shrink(x,idx)

        idx, x_c = order(x,f)
        f_best_e = f_of_x(x[idx[0]])
        if abs(f_best_e-f_best)< 10**(-8) and f_best!=f_best_e:
            print('optimization converged at iteration ',itr)
            break
    return x[idx[0]],f_of_x(x[idx[0]])

In [794]:
n = 10
N=10000
f_num = 2
display_on = 0
hi,lo,dim,f_of_x = init_optimization(f_num)
nelder_mead(N,n,dim,hi,lo, f_of_x, display_on)

Beale function in 2 dimension
Global minimum f(3,.5) = 0 for x,y in [-5,5]


(array([2.05121222, 0.30826812]), 0.5633888889030576)